In [16]:
import requests
import re
import time
import pandas as pd
from bs4 import BeautifulSoup
#import pymysql
from collections import Counter
import os
import random
import pandas as pd

'''
import smtplib
from smtplib import SMTP
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.header import Header
'''

def get_one_page(url):
    response = requests.get(url)
    print(response.status_code) 
    while response.status_code == 403:
        time.sleep(500 + random.uniform(0, 500))
        response = requests.get(url)
        print(response.status_code)
    print(response.status_code)
    if response.status_code == 200:
        return response.text

    return None




In [ ]:
os.chdir(sys.path[0])
url = 'https://arxiv.org/list/cs/pastweek?show=1000'
html = get_one_page(url)
soup = BeautifulSoup(html, features='html.parser')
content = soup.dl
date = soup.find('h3')
list_ids = content.find_all('a', title = 'Abstract')
list_title = content.find_all('div', class_ = 'list-title mathjax')
list_authors = content.find_all('div', class_ = 'list-authors')
list_subjects = content.find_all('div', class_ = 'list-subjects')
list_subjects_split = []
list_authors_split = []
list_title_split = []
# modify title
for title in list_title:
    title = title.text.split(': ', maxsplit=1)[1]
    list_title_split.append(title)

# modify subject
for subjects in list_subjects:
    subjects = subjects.text.split(': ', maxsplit=1)[1]
    subjects = subjects.replace('\n\n', '')
    subjects = subjects.replace('\n', '')
    subject_split = subjects.split('; ')
    list_subjects_split.append(subject_split)

# modify author
for authors in list_authors:
    authors = authors.text.split(':', maxsplit=1)[1]
    authors = authors.replace('\n\n', '')
    authors = authors.replace('\n', '')
    authors_splited = authors.split(', ')
    list_authors_split.append(authors_splited)
    

items = []
for i, paper in enumerate(zip(list_ids, list_title_split, list_authors_split, list_subject_split)):
    items.append([paper[0].text, paper[1], paper[2], paper[3]])
name = ['id', 'title', 'author', 'subject']
paper = pd.DataFrame(columns=name,data=items)
paper.to_csv('./'+time.strftime("%Y-%m-%d")+'_'+str(len(items))+'.csv')

In [ ]:
# wash


In [ ]:
'''subject split'''
subject_all = []
for subject_split in list_subject_split:
    for subject in subject_split:
        subject_all.append(subject)
subject_cnt = Counter(subject_all)
#print(subject_cnt)
subject_items = []
for subject_name, times in subject_cnt.items():
    subject_items.append([subject_name, times])
subject_items = sorted(subject_items, key=lambda subject_items: subject_items[1], reverse=True)
name = ['name', 'times']
subject_file = pd.DataFrame(columns=name,data=subject_items)
#subject_file = pd.DataFrame.from_dict(subject_cnt, orient='index')
subject_file.to_csv('/home/zzh/Code/Spider/paperspider/arxiv/sub_cnt/'+time.strftime("%Y-%m-%d")+'_'+str(len(items))+'.csv')
#subject_file.to_html('subject_file.html')    

In [ ]:
'''key_word1 selection'''
key_words = ['track', 'occlu', 'multiple object', 'multiple target', 'multi-object', 'multi-target', 'people', 'person', 'pedestrian', 'human', 'siam'] 
Key_words = ['MOT', 'SOT']
key_words2 = ['quantization', 'compress', 'prun']
Key_words2 = ['MOT']    
selected_papers = paper[paper['title'].str.contains(key_words[0], case=False)]
for key_word in key_words[1:]:
    selected_paper1 = paper[paper['title'].str.contains(key_word, case=False)]
    selected_papers = pd.concat([selected_papers, selected_paper1], axis=0)
for Key_word in Key_words[1:]:
    selected_paper1 = paper[paper['title'].str.contains(Key_word, case=True)]
    selected_papers = pd.concat([selected_papers, selected_paper1], axis=0)
selected_papers.to_csv('./'+time.strftime("%Y-%m-%d")+'_'+str(len(selected_papers))+'.csv')

In [ ]:
# email
'''
## key_word2 selection
selected_papers2 = paper[paper['title'].str.contains(key_words2[0], case=False)]
for key_word in key_words2[1:]:
    selected_paper1 = paper[paper['title'].str.contains(key_word, case=False)]
    selected_papers2 = pd.concat([selected_papers2, selected_paper1], axis=0)
for Key_word in Key_words2[1:]:
    selected_paper1 = paper[paper['title'].str.contains(Key_word, case=True)]
    selected_papers2 = pd.concat([selected_papers2, selected_paper1], axis=0)
selected_papers2.to_csv('/home/zzh/Code/Spider/paperspider/arxiv/selected2/'+time.strftime("%Y-%m-%d")+'_'+str(len(selected_papers2))+'.csv')
'''


'''send email'''
#selected_papers.to_html('email.html')
content = 'Today arxiv has {} new papers in CS area, and {} of them is about CV, {}/{} of them contain your keywords.\n\n'.format(len(list_title), subject_cnt['Computer Vision and Pattern Recognition (cs.CV)'], len(selected_papers), len(selected_papers2))
content += 'Ensure your keywords is ' + str(key_words) + ' and ' + str(Key_words) + '(case=True). \n\n'
content += 'This is your paperlist.Enjoy! \n\n'
for i, selected_paper in enumerate(zip(selected_papers['id'], selected_papers['title'], selected_papers['authors'], selected_papers['subject_split'])):
    #print(content1)
    content1, content2, content3, content4 = selected_paper
    content += '------------' + str(i+1) + '------------\n' + content1 + content2 + str(content4) + '\n'
    content1 = content1.split(':', maxsplit=1)[1]
    content += 'https://arxiv.org/abs/' + content1 + '\n\n'

content += 'Ensure your keywords2 is ' + str(key_words2) + ' and ' + str(Key_words2) + '(case=True). \n\n'
content += 'This is your paperlist.Enjoy! \n\n'
for i, selected_paper2 in enumerate(zip(selected_papers2['id'], selected_papers2['title'], selected_papers2['authors'], selected_papers2['subject_split'])):

    #print(content1)
    content1, content2, content3, content4 = selected_paper2
    content += '------------' + str(i+1) + '------------\n' + content1 + content2 + str(content4) + '\n'
    content1 = content1.split(':', maxsplit=1)[1]
    content += 'https://arxiv.org/abs/' + content1 + '\n\n'


content += 'Here is the Research Direction Distribution Report. \n\n'
for subject_name, times in subject_items:
    content += subject_name + '   ' + str(times) +'\n'
title = time.strftime("%Y-%m-%d") + ' you have {}+{} papers'.format(len(selected_papers), len(selected_papers2))
send_email(title, content)
freport = open('/home/zzh/Code/Spider/paperspider/arxiv/report/'+title+'.txt', 'w')
freport.write(content)
freport.close()

'''
# dowdload key_word selected papers
list_subject_split = []
if not os.path.exists('/home/zzh/Code/Spider/paperspider/arxiv/selected/'+time.strftime("%Y-%m-%d")):
    os.makedirs('/home/zzh/Code/Spider/paperspider/arxiv/selected/'+time.strftime("%Y-%m-%d"))
for selected_paper_id, selected_paper_title in zip(selected_papers['id'], selected_papers['title']):
    selected_paper_id = selected_paper_id.split(':', maxsplit=1)[1]
    selected_paper_title = selected_paper_title.split(':', maxsplit=1)[1]
    r = requests.get('https://arxiv.org/pdf/' + selected_paper_id) 
    while r.status_code == 403:
        time.sleep(500 + random.uniform(0, 500))
        r = requests.get('https://arxiv.org/pdf/' + selected_paper_id)
    selected_paper_id = selected_paper_id.replace(".", "_")
    pdfname = selected_paper_title.replace("/", "_")   #pdf名中不能出现/和：
    pdfname = pdfname.replace("?", "_")
    pdfname = pdfname.replace("\"", "_")
    pdfname = pdfname.replace("*","_")
    pdfname = pdfname.replace(":","_")
    pdfname = pdfname.replace("\n","")
    pdfname = pdfname.replace("\r","")
    print('/home/zzh/Code/Spider/paperspider/arxiv/selected/'+time.strftime("%Y-%m-%d")+'/%s %s.pdf'%(selected_paper_id, selected_paper_title))
    with open('/home/zzh/Code/Spider/paperspider/arxiv/selected/'+time.strftime("%Y-%m-%d")+'/%s %s.pdf'%(selected_paper_id,pdfname), "wb") as code:    
        code.write(r.content)
'''

In [4]:
# crasp subject to form table

os.chdir(sys.path[0])
#url = 'https://arxiv.org/list/cs/pastweek?show=1000'
url = 'https://arxiv.org'
html = get_one_page(url)
soup = BeautifulSoup(html, features='html.parser')
content = soup.dl


'''
list_ids = content.find_all('a', title = 'Abstract')
list_title = content.find_all('div', class_ = 'list-title mathjax')
list_authors = content.find_all('div', class_ = 'list-authors')
list_subjects = content.find_all('div', class_ = 'list-subjects')
list_subjects_split = []
list_authors_split = []
list_title_split = []
# modify title
for title in list_title:
    title = title.text.split(': ', maxsplit=1)[1]
    list_title_split.append(title)

# modify subject
for subjects in list_subjects:
    subjects = subjects.text.split(': ', maxsplit=1)[1]
    subjects = subjects.replace('\n\n', '')
    subjects = subjects.replace('\n', '')
    subject_split = subjects.split('; ')
    list_subjects_split.append(subject_split)

# modify author
for authors in list_authors:
    authors = authors.text.split(':', maxsplit=1)[1]
    authors = authors.replace('\n\n', '')
    authors = authors.replace('\n', '')
    authors_splited = authors.split(', ')
    list_authors_split.append(authors_splited)
    

items = []
for i, paper in enumerate(zip(list_ids, list_title_split, list_authors_split, list_subject_split)):
    items.append([paper[0].text, paper[1], paper[2], paper[3]])
name = ['id', 'title', 'author', 'subject']
paper = pd.DataFrame(columns=name,data=items)
paper.to_csv('./'+time.strftime("%Y-%m-%d")+'_'+str(len(items))+'.csv')'''

200
200


'\nlist_ids = content.find_all(\'a\', title = \'Abstract\')\nlist_title = content.find_all(\'div\', class_ = \'list-title mathjax\')\nlist_authors = content.find_all(\'div\', class_ = \'list-authors\')\nlist_subjects = content.find_all(\'div\', class_ = \'list-subjects\')\nlist_subjects_split = []\nlist_authors_split = []\nlist_title_split = []\n# modify title\nfor title in list_title:\n    title = title.text.split(\': \', maxsplit=1)[1]\n    list_title_split.append(title)\n\n# modify subject\nfor subjects in list_subjects:\n    subjects = subjects.text.split(\': \', maxsplit=1)[1]\n    subjects = subjects.replace(\'\n\n\', \'\')\n    subjects = subjects.replace(\'\n\', \'\')\n    subject_split = subjects.split(\'; \')\n    list_subjects_split.append(subject_split)\n\n# modify author\nfor authors in list_authors:\n    authors = authors.text.split(\':\', maxsplit=1)[1]\n    authors = authors.replace(\'\n\n\', \'\')\n    authors = authors.replace(\'\n\', \'\')\n    authors_splited = auth

In [16]:
psubjects = soup.find_all('h2')
print(psubjects)

AttributeError: ResultSet object has no attribute 'dl'. You're probably treating a list of items like a single item. Did you call find_all() when you meant to call find()?

In [5]:
#def getSubjects(pSubjectNameAbbr, )
pSubjectAbbr = 'cs'
subjects = soup.find_all('a', id=re.compile(r'cs.[A-Z]{2}') ) #, class_=re.compile(r'cs.[A-Z]{2}')
#print(subjects)
for s in subjects:
    print(s.text)

Artificial Intelligence
Computation and Language
Computational Complexity
Computational Engineering, Finance, and Science
Computational Geometry
Computer Science and Game Theory
Computer Vision and Pattern Recognition
Computers and Society
Cryptography and Security
Data Structures and Algorithms
Databases
Digital Libraries
Discrete Mathematics
Distributed, Parallel, and Cluster Computing
Emerging Technologies
Formal Languages and Automata Theory
General Literature
Graphics
Hardware Architecture
Human-Computer Interaction
Information Retrieval
Information Theory
Logic in Computer Science
Machine Learning
Mathematical Software
Multiagent Systems
Multimedia
Networking and Internet Architecture
Neural and Evolutionary Computing
Numerical Analysis
Operating Systems
Other Computer Science
Performance
Programming Languages
Robotics
Social and Information Networks
Software Engineering
Sound
Symbolic Computation
Systems and Control


In [10]:
# form .csv
def saveCSV(subjects, psubjectId, psubjectName):
    items = []
    for i, s in enumerate(subjects):
        items.append([s.text, psubjectId])
    name = ['subject', 'psubject']
    paper = pd.DataFrame(columns=name,data=items,index=None)
    paper.to_csv('./'+psubjectName+'_'+str(len(items))+'.csv')

In [11]:
saveCSV(subjects, 1, 'Computer Science')

In [13]:
pdSubjects = pd.read_csv('./subject.csv')
pdSubjects.head(5)

,id,name,pid
0,1,Computer Science,0
1,6,Artificial Intelligence,1
2,7,Computation and Language,1
3,8,Computational Complexity,1
4,9,"Computational Engineering, Finance, and Science",1


In [14]:
os.chdir(sys.path[0])
url = 'https://arxiv.org/list/cs/new'
html = get_one_page(url)
soup = BeautifulSoup(html, features='html.parser')
content = soup.dl

200
200


In [40]:
# save paper

date = soup.find('h3')
list_ids = content.find_all('a', title = 'Abstract')
list_title = content.find_all('div', class_ = 'list-title mathjax')
list_authors = content.find_all('div', class_ = 'list-authors')
list_subjects = content.find_all('div', class_ = 'list-subjects')
list_subjects_split = []
list_authors_split = []
list_title_split = []
list_id = []
list_url_split = []
# modify url
for i in list_ids:
    
    i = i.text.split(':', maxsplit=1)[1]
    i.strip()
    list_id.append(int(i.split('.')[1]))
    list_url_split.append('arxiv.org/abs/'+i)

# modify title
for title in list_title:
    title = title.text.split(': ', maxsplit=1)[1]
    title = title.replace('\n', '')
    list_title_split.append(title)

# modify subject
for subjects in list_subjects:
    subjects = subjects.text.split(': ', maxsplit=1)[1]
    subjects = subjects.replace('\n\n', '')
    subjects = subjects.replace('\n', '')
    subject_split = subjects.split('; ')
    subject_split = [i.split(' (')[0] for i in subject_split]
    #print(subject_split)
    ids = pdSubjects[pdSubjects['name'].isin(subject_split)]['id']
    #print(ids)
    ids = list(ids)
    list_subjects_split.append(ids)

# modify author
for authors in list_authors:
    authors = authors.text.split(':', maxsplit=1)[1]
    authors = authors.replace('\n\n', '')
    authors = authors.replace('\n', '')
    #authors_splited = authors.split(', ')
    #authors.replace(re.compile(r'\(\w+\)'), '')
    #list_authors_split.append(authors_splited)
    authors = authors.replace(', ', ',')
    list_authors_split.append(authors)
    

items = []
for i, paper in enumerate(zip(list_id, list_url_split, list_title_split, list_authors_split, list_subjects_split)):
    items.append([paper[0], paper[1], paper[2], paper[3], paper[4]])
name = ['id', 'url', 'title', 'author', 'subject']
paper = pd.DataFrame(columns=name,data=items)
paperCSVName = 
paper.to_csv('./paper_'+time.strftime("%Y-%m-%d")+'_'+str(len(items))+'.csv')
paper.head(5)

,id,url,title,author,subject
0,2380,arxiv.org/abs/2005.02380,Bit-Interleaved Coded Multiple Beamforming wit...,"Sadjad Sedighi,Ender Ayanoglu",[27]
1,2384,arxiv.org/abs/2005.02384,Compositionality of the MSO+U Logic,Paweł Parys,[28]
2,2387,arxiv.org/abs/2005.02387,SurvLIME-Inf: A simplified modification of Sur...,"Lev V. Utkin,Maxim S. Kovalev,Ernest M. Kasimov",[29]
3,2389,arxiv.org/abs/2005.02389,Jointly Sparse Support Recovery via Deep Auto-...,"Wanqing Zhang,Shuaichao Li,Ying Cui",[27]
4,2390,arxiv.org/abs/2005.02390,Mechanism Design and Blockchains,"Akaki Mamageishvili,Jan Christoph Schlegel",[11]


In [42]:
paper_subject_items = []
for item in items:
    for subject in item[4]:
        paper_subject_items.append([item[0], subject])
paper_subject = pd.DataFrame(columns=['paper_id', 'subject_id'], data=paper_subject_items)
paper_subject.to_csv('./paper_subject_'+time.strftime("%Y-%m-%d")+'_'+str(len(items))+'.csv', index=None)
    

In [37]:
dfPaper = pd.read_csv('./paper.csv')
dfPaper.head()

,id,title
0,1892,Bit-Interleaved Coded Multiple Beamforming wit...
1,1893,Compositionality of the MSO+U Logic
2,1894,SurvLIME-Inf: A simplified modification of Sur...
3,1895,Jointly Sparse Support Recovery via Deep Auto-...
4,1896,Mechanism Design and Blockchains


In [34]:
title = ['paper_id', 'subject_id']
items = []
for i in dfPaper:
    for j in i['subject']

,Unnamed: 0,url,title,author,subject
1,1,arxiv.org/abs/2005.02384,Compositionality of the MSO+U Logic,Paweł Parys,[28]
2,2,arxiv.org/abs/2005.02387,SurvLIME-Inf: A simplified modification of Sur...,"Lev V. Utkin,Maxim S. Kovalev,Ernest M. Kasimov",[29]
3,3,arxiv.org/abs/2005.02389,Jointly Sparse Support Recovery via Deep Auto-...,"Wanqing Zhang,Shuaichao Li,Ying Cui",[27]
4,4,arxiv.org/abs/2005.02390,Mechanism Design and Blockchains,"Akaki Mamageishvili,Jan Christoph Schlegel",[11]
5,5,arxiv.org/abs/2005.02392,Deep Lagrangian Constraint-based Propagation i...,"Matteo Tiezzi,Giuseppe Marra,Stefano Melacci,M...",[29]


In [45]:
# activity
os.chdir(sys.path[0])
url = 'http://www.allconferences.com/search/index/Category__parent_id:12290/Conference__start_date__from:05-01-2020/Conference__start_date__to:11-01-2020/showLastConference:1/'
html = get_one_page(url)
soup = BeautifulSoup(html, features='html.parser')
content = soup.dl

200
200


In [84]:
#print(soup)
conferences = soup.find_all('div', id=re.compile(r'conference_\d+'))
'''
begins = soup.find_all('span', class_='begin_txt')
ends = soup.find_all('span', class_='end_txt')
namesAndUrls = soup.find_all('a', href = re.compile(r'http://www\.allconferences\.com[\s\S]*'))
'''#date = soup.find('h3')
#print(namesAndUrls)
def timeTransfer(origin):
    tmp = time.strptime(origin, "%b %d, %Y")
    return time.strftime("%Y-%m-%d %H:%M:%S", tmp)
activities = []
for c in conferences:
    beginTime = c.find('span', class_='begin_txt').text.split('\n',maxsplit=2)[1]
    beginTime = timeTransfer(beginTime)
    endTime = c.find('span', class_='end_txt').text.split('\r\n',maxsplit=1)[1].strip()
    #print(beginTime)
    endTime = timeTransfer(endTime)
    url = c.find('h2').find('a')
    #print(url)
    name = url.text
    if name == None:
        continue
    #print(name)
    url = url.get('href')
    #name = url.text
    #print(beginTime, endTime, url, name)
    print("===========")
    activities.append([name, url, beginTime, endTime])



In [85]:

activity = pd.DataFrame(columns=['name', 'url', 'beginTime', 'endTime'],data=activities)
activity.to_csv('./activity_'+time.strftime("%Y-%m-%d")+'_'+str(len(activities))+'.csv',index=None)
activity.head(5)

,name,url,beginTime,endTime
0,IEEE--2020 The 5th International Conference on...,http://www.allconferences.com/c/ieee-2020-the-...,2020-05-08 00:00:00,2020-05-11 00:00:00
1,ACM--2020 the 4th Int. Conference on Innovatio...,http://www.allconferences.com/c/acm-2020-the-4...,2020-05-08 00:00:00,2020-05-11 00:00:00
2,2020 The 7th International Conference on Digit...,http://www.allconferences.com/c/2020-the-7th-i...,2020-05-10 00:00:00,2020-05-12 00:00:00
3,2020 International Conference on Wireless Comm...,http://www.allconferences.com/c/2020-internati...,2020-05-13 00:00:00,2020-05-15 00:00:00
4,2020 World Conference on Computing and Communi...,http://www.allconferences.com/c/2020-world-con...,2020-05-13 00:00:00,2020-05-15 00:00:00
